In [ ]:



DEBUG_FLAG=True
GOAL = 10

'''

 debug message : turn DEBUG_FLAG off to suppress messages

'''
def debug (msg,obj):
  if (DEBUG_FLAG):
    print(msg,end="")
    print(obj)


'''

 goalp : check if it is a leaf (leaf value > GOAL, then goal found)

'''
def goalp (node):

  if (not isinstance(node,int)):
    # not a leaf
    debug("  goalp: not a leaf node","")
    return False

  else:
    # leaf node
    if (node > GOAL):
      debug("  goalp: MATCH","")
      return node
    else:
      debug("  goalp: NO match","")
      return False

# test:
# goalp([ [ 1 ,2 ] , 3] )
# goalp(11)


'''

 expand : given node, expand into childern, and return list containing children

'''
def expand (node):

  if (isinstance(node,int)):
     return []
  else:
     '''
     If "node" = [ a, b, c ]
     then expanded items are a, b, c.
     Putting a, b, c in a list gives [ a, b, c ], which is the same as "node".
     '''
     return node

# test:
# expand([ 2 , [3 , 4 ], 5])
# expand(2)

'''

 append : append two lists a and b

'''
def append_list(a, b):
  ret = a
  ret.extend(b)
  return ret



In [ ]:
'''

  dfs : iterative version

'''
def dfs (node):

  # put node in empty nodelist
  node_list = [node]
  debug("Start: ", node_list)

  # MAIN LOOP: while node_list is not empty
  while (node_list):

    # A. visit: pop first node from list
    cur_node = node_list.pop(0)
    debug("Visiting: ", cur_node)

    # B. goal check
    if (goalp(cur_node)):

      # 1 goal found
      print("  MATCH = "+str(cur_node))
      return cur_node

    else:
      # 2 further explore

      # 2.1: expand node
      expanded = expand(cur_node)
      debug("  Expanded=", expanded)

      # 2.2: enqueue (change here to change search behavior)

      # for dfs:
      node_list = append_list(expanded, node_list)
      # for bfs: change to
      #node_list = append_list(node_list, expanded)

    # C. print current node_list
    debug("  New node list=",node_list)
    debug("","")



In [ ]:
# test: change the tree or change the goal (set GOAL at the top)
dfs([2 , [3 ,[7, 11 ]], [9, 20], 5])

Start: [[2, [3, [7, 11]], [9, 20], 5]]
Visiting: [2, [3, [7, 11]], [9, 20], 5]
  goalp: not a leaf node
  Expanded=[2, [3, [7, 11]], [9, 20], 5]
  New node list=[2, [3, [7, 11]], [9, 20], 5]

Visiting: 2
  goalp: NO match
  Expanded=[]
  New node list=[[3, [7, 11]], [9, 20], 5]

Visiting: [3, [7, 11]]
  goalp: not a leaf node
  Expanded=[3, [7, 11]]
  New node list=[3, [7, 11], [9, 20], 5]

Visiting: 3
  goalp: NO match
  Expanded=[]
  New node list=[[7, 11], [9, 20], 5]

Visiting: [7, 11]
  goalp: not a leaf node
  Expanded=[7, 11]
  New node list=[7, 11, [9, 20], 5]

Visiting: 7
  goalp: NO match
  Expanded=[]
  New node list=[11, [9, 20], 5]

Visiting: 11
  goalp: MATCH
  MATCH = 11


11

In [ ]:
###########################################################################
# NODE data structure
#
# Below are some ideas on how to make the simple example above more general
# - use adjacency matrix to define graph, and conduct search on the graph
# - each cell in the matrix is the distance
# - column -> row connectivity
###########################################################################

# EXAMPLE search problem

# 1. adjacency matrix : states 0, 1, 2, 3.
#    - assume Init =0 , Goal = 3.
#    - cell values indicate the cost of each move
MAT = [[0, 20, 0,  10, 60, 0, 0],
       [0,  0,  30, 0, 50, 10, 0],
       [0, 0, 0, 0, 10, 0, 40],
       [0,  0, 60,  0, 0, 40, 0],
       [0, 0, 0, 0, 0, 0, 10],
       [0, 0, 10, 0, 0, 0, 45],
       [0, 0, 0, 0, 0, 0, 0]]
'''
 The search graph looks like this:

 (0) --10--> (2) --20--> (3)
  |     _10__/            ^
  30  /                   |
  |  |                    |
  v  v                   /
  (1)_______5___________/

'''

# 2. heuristic values : states 0, 1, 2, 3
H = [55, 35, 17, 66, 9, 27, 0]


'''

  NODE CLASS

  - assumes MAT and H are defined as global variables (see above)
  - support creation of new start node, expand, and printing node and node list.

'''
class Node:

  def __init__(self, state):
    '''
    initialize a node
    '''
    self.state = state
    self.h = H[state]
    self.cost = 0
    self.path = str(state)

  def move(self, child_state):
    '''
    make one move to child_state
    - returns resulting child node
    - do all the book-keeping
    '''
    edge_cost = MAT[self.state][child_state]
    if (edge_cost>0):
      child       = Node(child_state)
      child.cost = self.cost + edge_cost
      child.path  = self.path+"->"+str(child_state)
      return child
    else:
      return []

  def expand(self):
    '''
    expand node and do all the book-keeping
    - returns node list
    '''

    node_list = [ ]

    for child_state in range(0,len(H)):
       if (MAT[self.state][child_state]>0):
         child_node = self.move(child_state)
         node_list.extend([child_node])

    self.sort(node_list, "A*")

    return node_list

  def print(self):
    '''
    print out node data structure
    '''
    print("{Node: state="+str(self.state)+", cost="+str(self.cost)+", h="+str(self.h)+", path=("+self.path+")} ",end="")

  def print_list(node_list):
    '''
    print out a node list
    '''
    print("[ ",end="")

    for n in node_list:
      Node.print(n)
      print(" ",end="")

    print("]")

  def sort(self, node_list, eval_fn):
    '''
    For informed search, you need to sort the list using a certain evaluation function
    - the current implementation is a place holder: it returns an unsorted list
    '''
    if eval_fn == "greedy":
      print("Greedy Best first")
      node_list.sort(key=lambda x : x.h)
    else :
      print("A*")
      node_list.sort(key= lambda x : (x.h + x.cost))
    return node_list


In [ ]:
# test node class

print("Testing Node class")
node = Node(0)
node.print()
print("")
Node.print_list([node, node, node])
print("\n")

# test move(): Move 0 -> 1

print("Testing Node class : move function")
node = Node(0)
child = node.move(1)
child.print()
print("\n\n")

# test expand() : expand from state 0

print("Testing Node class : expand function")
node = Node(0)
expanded_node_list = node.expand()
Node.print_list(expanded_node_list)
print("\n")

print("Testing Node class : expand function - next level")
for n in expanded_node_list:
  grand_children = n.expand()
  Node.print_list(grand_children)

Testing Node class
{Node: state=0, cost=0, h=55, path=(0)} 
[ {Node: state=0, cost=0, h=55, path=(0)}  {Node: state=0, cost=0, h=55, path=(0)}  {Node: state=0, cost=0, h=55, path=(0)}  ]


Testing Node class : move function
{Node: state=1, cost=20, h=35, path=(0->1)} 


Testing Node class : expand function
A*
[ {Node: state=1, cost=20, h=35, path=(0->1)}  {Node: state=4, cost=60, h=9, path=(0->4)}  {Node: state=3, cost=10, h=66, path=(0->3)}  ]


Testing Node class : expand function - next level
A*
[ {Node: state=5, cost=30, h=27, path=(0->1->5)}  {Node: state=2, cost=50, h=17, path=(0->1->2)}  {Node: state=4, cost=70, h=9, path=(0->1->4)}  ]
A*
[ {Node: state=6, cost=70, h=0, path=(0->4->6)}  ]
A*
[ {Node: state=5, cost=50, h=27, path=(0->3->5)}  {Node: state=2, cost=70, h=17, path=(0->3->2)}  ]


In [ ]:
def astar(start_state, goal_state):
    open_list = []
    node = Node(start_state)
    while node.state != goal_state:
      open_list.extend(node.expand())
      open_list.sort(key= lambda x : (x.h + x.cost))
      Node.print_list(open_list)
      node = node.move(open_list[0].state)
      open_list.pop(0)

    print("Cost =", node.cost)
    print("Path = ", node.path)

astar(0,6)


A*
[ {Node: state=1, cost=20, h=35, path=(0->1)}  {Node: state=4, cost=60, h=9, path=(0->4)}  {Node: state=3, cost=10, h=66, path=(0->3)}  ]
A*
[ {Node: state=5, cost=30, h=27, path=(0->1->5)}  {Node: state=2, cost=50, h=17, path=(0->1->2)}  {Node: state=4, cost=60, h=9, path=(0->4)}  {Node: state=3, cost=10, h=66, path=(0->3)}  {Node: state=4, cost=70, h=9, path=(0->1->4)}  ]
A*
[ {Node: state=2, cost=40, h=17, path=(0->1->5->2)}  {Node: state=2, cost=50, h=17, path=(0->1->2)}  {Node: state=4, cost=60, h=9, path=(0->4)}  {Node: state=6, cost=75, h=0, path=(0->1->5->6)}  {Node: state=3, cost=10, h=66, path=(0->3)}  {Node: state=4, cost=70, h=9, path=(0->1->4)}  ]
A*
[ {Node: state=4, cost=50, h=9, path=(0->1->5->2->4)}  {Node: state=2, cost=50, h=17, path=(0->1->2)}  {Node: state=4, cost=60, h=9, path=(0->4)}  {Node: state=6, cost=75, h=0, path=(0->1->5->6)}  {Node: state=3, cost=10, h=66, path=(0->3)}  {Node: state=4, cost=70, h=9, path=(0->1->4)}  {Node: state=6, cost=80, h=0, path=(

In [ ]:
def greedybestfs(start_state, goal_state):
    open_list = []
    node = Node(start_state)
    while node.state != goal_state:
      open_list.extend(node.expand())
      open_list.sort(key= lambda x : (x.h))
      Node.print_list(open_list)
      node = node.move(open_list[0].state)
      open_list.pop(0)

    print("Cost =", node.cost)
    print("Path = ", node.path)

greedybestfs(0,6)


A*
[ {Node: state=4, cost=60, h=9, path=(0->4)}  {Node: state=1, cost=20, h=35, path=(0->1)}  {Node: state=3, cost=10, h=66, path=(0->3)}  ]
A*
[ {Node: state=6, cost=70, h=0, path=(0->4->6)}  {Node: state=1, cost=20, h=35, path=(0->1)}  {Node: state=3, cost=10, h=66, path=(0->3)}  ]
Cost = 70
Path =  0->4->6
